# Entre lavouras e florestas: Otimizando recursos com sustentabilidade nas ciências agrárias
## Prof. Mayron César O. Moreira (DC/UFLA)
### Congresso Unificado da UFLA 2025
*11 de novembro de 2025*

*Dados de entrada*:

| Talhão | Volume (m³) | Lucro (R$ mil) | Risco ambiental (0–10) |
|--------|-------------|----------------|------------------------|
| 1      | 90          | 45             | 8                      |
| 2      | 120         | 55             | 9                      |
| 3      | 70          | 35             | 3                      |
| 4      | 110         | 50             | 5                      |
| 5      | 100         | 47             | 6                      |
| 6      | 80          | 38             | 2                      |

In [9]:
talhao = {
    1: {'volume': 90, 'lucro': 45000, 'risco': 8},
    2: {'volume': 120, 'lucro': 55000, 'risco': 9},
    3: {'volume': 70, 'lucro': 35000, 'risco': 3},
    4: {'volume': 110, 'lucro': 50000, 'risco': 5},
    5: {'volume': 100, 'lucro': 47000, 'risco': 6},
    6: {'volume': 80, 'lucro': 38000, 'risco': 2},
}

In [10]:
import gurobipy as gp
from gurobipy import GRB

#  Variáveis de decisão

- $x_i \in \{0,1\}$: 1 se o talhão $i$ for selecionado para cultivo, 0 caso contrário, para $i = 1, 2, \ldots, 6$.

In [11]:
modelo = gp.Model("dinamica1")

In [12]:
x = modelo.addVars(talhao.keys(), vtype=GRB.BINARY, name="x")

* *Função objetivo:* Z = 0,7 × (Lucro) − 0,3 × (Risco)

In [13]:
modelo.setObjective(
    sum(0.7 * talhao[i]['lucro'] * x[i] - 0.3 * talhao[i]['risco'] * x[i] for i in talhao),
    GRB.MAXIMIZE
)

*Restrições:*  
1. **Três talhões devem ser selecionados:** $\sum_{i=1}^{6} x_i = 3$

In [14]:
# Restrições
modelo.addConstr(
    sum(x[i] for i in talhao) == 3,
    "tres_talhoes_selecionados"
)

<gurobi.Constr *Awaiting Model Update*>

In [15]:
modelo.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-13620H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1 rows, 6 columns and 6 nonzeros
Model fingerprint: 0xf02739ab
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective 94494.000000
Presolve removed 1 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 106394 94494 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.063940000000e+05, best bound 1.063940000000e+05, gap 0.0000%


🧮 *Resultados*

In [16]:
import pandas as pd

resultados = pd.DataFrame({
    'Talhão': list(talhao.keys()),
    'Selecionado': [int(x[i].X) for i in talhao],
    'Volume (m³)': [talhao[i]['volume'] for i in talhao],
    'Lucro por ha (R$)': [talhao[i]['lucro'] for i in talhao],
    'Risco ambiental (0–10)': [talhao[i]['risco'] for i in talhao],
})  

resultados["Lucro Total (R$)"] = resultados['Selecionado'] * resultados['Lucro por ha (R$)']
resultados["Volume Total (m³)"] = resultados['Selecionado'] * resultados['Volume (m³)']
resultados = resultados[resultados['Selecionado'] == 1].reset_index(drop=True)
resultados

,Talhão,Selecionado,Volume (m³),Lucro por ha (R$),Risco ambiental (0–10),Lucro Total (R$),Volume Total (m³)
0,2,1,120,55000,9,55000,120
1,4,1,110,50000,5,50000,110
2,5,1,100,47000,6,47000,100
